<a href="https://colab.research.google.com/github/cvbrandoe/coursHNS/blob/main/notebooks/ENC_HNS_Geo_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geoparsing avec spaCy et Stanza : difficultés et cas Limites

Ce notebook explore les outils TAL pour le geoparsing des textes en mettant l'accent sur leurs forces, faiblesses et les cas problématiques.

Objectifs :
- Comparer spaCy (https://spacy.io/) et Stanza (https://stanfordnlp.github.io/stanza/) pour la reconnaissance automatisée de noms de lieux
- Identifier les difficultés spécifiques aux textes historiques/littéraires
- Développer des stratégies pour améliorer la détection


## Installation des bibliothèques

In [1]:
!pip install spacy stanza pandas matplotlib seaborn
!python -m spacy download fr_core_news_md
!python -m spacy download fr_core_news_lg

# !python -m spacy download en_core_web_sm

# %%
import spacy
import stanza
import pandas as pd
from collections import Counter, defaultdict
import re
import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 8.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.8/571.8 MB 2.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## Chargement et configuration des modèles


In [2]:
print("Chargement des modèles TAL...")

# spaCy - modèle français moyen
nlp_spacy_md = spacy.load("fr_core_news_md")
print("✓ spaCy FR (md) chargé")

# spaCy - modèle français large (optionnel, plus précis)
try:
    nlp_spacy_lg = spacy.load("fr_core_news_lg")
    print("✓ spaCy FR (lg) chargé")
except:
    print("⚠ spaCy FR (lg) non disponible - utilisez 'md' uniquement")
    nlp_spacy_lg = None

# Stanza - téléchargement automatique si nécessaire
try:
    nlp_stanza = stanza.Pipeline('fr', processors='tokenize,ner', verbose=False)
    print("✓ Stanza FR chargé")
except:
    print("Téléchargement de Stanza FR...")
    stanza.download('fr')
    nlp_stanza = stanza.Pipeline('fr', processors='tokenize,ner', verbose=False)
    print("✓ Stanza FR chargé")

Chargement des modèles TAL...
✓ spaCy FR (md) chargé
✓ spaCy FR (lg) chargé
✓ Stanza FR chargé


## Cas difficiles : Typologie des problèmes


In [3]:
# Corpus de cas problématiques
cas_problematiques = {

    "1. Ambiguïté avec noms communs": [
        "Le roi de France convoqua les états généraux à Versailles.",
        "L'Église catholique possédait des terres en Provence.",
        "Le pape résidait à Avignon au XIVe siècle."
    ],

    "2. Noms historiques vs modernes": [
        "Constantin déplaça la capitale à Byzance, rebaptisée Constantinople.",
        "Les troupes napoléoniennes entrèrent dans Petrograd en hiver.",  # Anachronisme volontaire
        "Le traité fut signé à Ratisbonne, aujourd'hui Regensburg."
    ],

    "3. Lieux disparus ou fictifs": [
        "Les habitants de Pompéi ignoraient le danger du Vésuve.",
        "L'Atlantide aurait été située au-delà des Colonnes d'Hercule.",
        "Le royaume de Cocagne attirait les rêveurs médiévaux."
    ],

    "4. Micro-toponymie et lieux locaux": [
        "Il habitait rue de la Paix dans le quartier du Marais.",
        "Le moulin de Valmy devint le symbole de la bataille.",
        "L'abbaye de Cluny rayonnait sur toute la Bourgogne."
    ],

    "5. Homonymies géographiques": [
        "Paris envoya un ambassadeur à Londres.",
        "Il voyagea de Paris au Caire via Alexandrie.",
        "Le duc de Bourgogne rejoignit le duc d'Orléans."  # Titres vs lieux
    ],

    "6. Dérivés et adjectifs": [
        "Les Parisiens manifestèrent contre les Versaillais.",
        "L'art florentin influença toute l'Europe.",
        "Les soldats prussiens affrontèrent les Français."
    ],

    "7. Expressions figées avec toponymes": [
        "C'est un véritable château en Espagne.",
        "Il cherche midi à quatorze heures à Rome.",
        "Tous les chemins mènent à Rome."
    ],

    "8. Orthographe ancienne": [
        "Le roy partit de Versailles pour Saint-Germain-en-Laye.",
        "Les navires mouillèrent à Venise et Constantinoble.",
        "Nostre-Dame de Paris fut achevée au XIIIe siècle."
    ]
}

## Analyse REN avec spaCy et Stanza


In [4]:
def analyser_avec_spacy(texte, modele):
    """Extrait les entités géographiques avec spaCy"""
    doc = modele(texte)
    entites = []

    for ent in doc.ents:
        if ent.label_ in ["LOC", "GPE"]:  # Location, Geo-Political Entity
            entites.append({
                'texte': ent.text,
                'type': ent.label_,
                'debut': ent.start_char,
                'fin': ent.end_char
            })

    return entites

def analyser_avec_stanza(texte):
    """Extrait les entités géographiques avec Stanza"""
    doc = nlp_stanza(texte)
    entites = []

    for sent in doc.sentences:
        for ent in sent.ents:
            if ent.type == "LOC":
                entites.append({
                    'texte': ent.text,
                    'type': ent.type,
                    'debut': ent.start_char,
                    'fin': ent.end_char
                })

    return entites

## Fonction de comparaison spaCy vs Stanza

In [5]:
def comparer_outils(texte, afficher=True):
    """Compare les résultats de spaCy et Stanza sur un texte"""

    # Analyse avec les différents outils
    spacy_md = analyser_avec_spacy(texte, nlp_spacy_md)
    stanza = analyser_avec_stanza(texte)

    if nlp_spacy_lg:
        spacy_lg = analyser_avec_spacy(texte, nlp_spacy_lg)
    else:
        spacy_lg = []

    # Extraction des textes pour comparaison
    set_spacy_md = {e['texte'] for e in spacy_md}
    set_spacy_lg = {e['texte'] for e in spacy_lg} if spacy_lg else set()
    set_stanza = {e['texte'] for e in stanza}

    if afficher:
        print(f"\nTexte analysé : {texte}\n")
        print(f"spaCy (md)  : {set_spacy_md if set_spacy_md else '∅'}")
        if nlp_spacy_lg:
            print(f"spaCy (lg)  : {set_spacy_lg if set_spacy_lg else '∅'}")
        print(f"Stanza      : {set_stanza if set_stanza else '∅'}")

        # Analyse des différences
        tous_lieux = set_spacy_md | set_spacy_lg | set_stanza

        if len(tous_lieux) > 0:
            print("\n📊 Accord entre outils :")
            for lieu in tous_lieux:
                outils = []
                if lieu in set_spacy_md:
                    outils.append("spaCy-md")
                if lieu in set_spacy_lg:
                    outils.append("spaCy-lg")
                if lieu in set_stanza:
                    outils.append("Stanza")

                accord = len(outils)
                symbole = "✓✓✓" if accord == 3 else "✓✓" if accord == 2 else "✓"
                print(f"  {symbole} '{lieu}' → {', '.join(outils)}")

    return {
        'spacy_md': spacy_md,
        'spacy_lg': spacy_lg,
        'stanza': stanza
    }


## Analyse systématique des cas difficiles

In [6]:
resultats_globaux = []

for categorie, exemples in cas_problematiques.items():
    print(f"\n{'='*70}")
    print(f"CATÉGORIE : {categorie}")
    print(f"{'='*70}")

    for i, texte in enumerate(exemples, 1):
        print(f"\n--- Exemple {i} ---")
        resultats = comparer_outils(texte, afficher=True)

        resultats_globaux.append({
            'categorie': categorie,
            'texte': texte,
            'spacy_md_count': len(resultats['spacy_md']),
            'spacy_lg_count': len(resultats['spacy_lg']),
            'stanza_count': len(resultats['stanza']),
        })

# %% [markdown]
## 5. Statistiques comparatives

# %%
df_resultats = pd.DataFrame(resultats_globaux)

print("\n" + "="*70)
print("STATISTIQUES GLOBALES")
print("="*70)

print("\nLieux détectés par outil :")
print(f"  spaCy (md) : {df_resultats['spacy_md_count'].mean():.2f}")
print(f"  spaCy (lg) : {df_resultats['spacy_lg_count'].mean():.2f}")
print(f"  Stanza     : {df_resultats['stanza_count'].mean():.2f}")

print("\nNombre moyen de lieux détectés par outil :")
print(f"  spaCy (md) : {df_resultats['spacy_md_count'].mean():.2f}")
print(f"  spaCy (lg) : {df_resultats['spacy_lg_count'].mean():.2f}")
print(f"  Stanza     : {df_resultats['stanza_count'].mean():.2f}")

print("\nDétection par catégorie :")
stats_categories = df_resultats.groupby('categorie')[['spacy_md_count', 'stanza_count']].mean()
print(stats_categories)



CATÉGORIE : 1. Ambiguïté avec noms communs

--- Exemple 1 ---

Texte analysé : Le roi de France convoqua les états généraux à Versailles.

spaCy (md)  : {'Versailles'}
spaCy (lg)  : {'Versailles'}
Stanza      : {'Versailles', 'France'}

📊 Accord entre outils :
  ✓✓✓ 'Versailles' → spaCy-md, spaCy-lg, Stanza
  ✓ 'France' → Stanza

--- Exemple 2 ---

Texte analysé : L'Église catholique possédait des terres en Provence.

spaCy (md)  : {'Provence'}
spaCy (lg)  : {'Provence'}
Stanza      : {'Provence'}

📊 Accord entre outils :
  ✓✓✓ 'Provence' → spaCy-md, spaCy-lg, Stanza

--- Exemple 3 ---

Texte analysé : Le pape résidait à Avignon au XIVe siècle.

spaCy (md)  : {'XIVe siècle', 'Avignon'}
spaCy (lg)  : {'XIVe', 'Avignon'}
Stanza      : {'Avignon'}

📊 Accord entre outils :
  ✓ 'XIVe' → spaCy-lg
  ✓ 'XIVe siècle' → spaCy-md
  ✓✓✓ 'Avignon' → spaCy-md, spaCy-lg, Stanza

CATÉGORIE : 2. Noms historiques vs modernes

--- Exemple 1 ---

Texte analysé : Constantin déplaça la capitale à Byzance, 

## Problèmes spécifiques et solutions


1️⃣  FAUX NÉGATIFS (lieux non détectés)
   Causes :
   - Orthographe ancienne : "Nostre-Dame", "Constantinoble"
   - Micro-toponymie : rues, quartiers, petits villages
   - Noms composés mal segmentés
   
   Solutions :
   ✓ Pré-traitement : normalisation orthographique
   ✓ Dictionnaires personnalisés de gazetteer
   ✓ Règles basées sur les motifs (regex)
   ✓ Fine-tuning des modèles sur corpus historique

2️⃣  FAUX POSITIFS (non-lieux détectés)
   Causes :
   - Noms communs : "Église", "France" (dans "roi de France")
   - Titres nobiliaires : "duc d'Orléans"
   - Expressions figées : "château en Espagne"
   
   Solutions :
   ✓ Post-traitement avec listes de stop-words
   ✓ Analyse syntaxique (dépendances)
   ✓ Vérification contextuelle (POS-tagging)
   ✓ Filtrage par base de données géographique

3️⃣  AMBIGUÏTÉ RÉFÉRENTIELLE
   Causes :
   - Homonymies : Paris (France/Texas), Orléans (lieu/titre)
   - Variation temporelle : Constantinople/Istanbul
   
   Solutions :
   ✓ Désambiguïsation par contexte
   ✓ Bases de données historiques (Pleiades, GeoNames Historical)
   ✓ Fenêtre contextuelle élargie
   ✓ Heuristiques géographiques (proximité)

4️⃣  GENTILÉS ET DÉRIVÉS
   Causes :
   - "Parisiens", "florentin", "prussiens" non reconnus comme lieux
   
   Solutions :
   ✓ Lemmatisation : Parisiens → Paris
   ✓ Table de correspondance gentilés/lieux
   ✓ Expansion du modèle NER

5️⃣  PERFORMANCE DES OUTILS
   Observations :
   - spaCy (lg) > spaCy (md) en précision mais plus lent
   - Stanza parfois plus conservateur (moins de faux positifs)
   - Aucun outil parfait : complémentarité nécessaire
   
   Solutions :
   ✓ Approche ensembliste (vote/intersection)
   ✓ Validation manuelle sur échantillon
   ✓ Création de gold standard pour évaluation

## Amélioration avec des règles personnalisées (spaCy)


In [7]:
from spacy.matcher import Matcher
from spacy.tokens import Span

# Création d'un matcher pour patterns spécifiques
matcher = Matcher(nlp_spacy_md.vocab)

# Pattern 1 : "rue de/du/des [Nom]"
pattern_rue = [
    {"LOWER": "rue"},
    {"LOWER": {"IN": ["de", "du", "des", "de la"]}},
    {"POS": {"IN": ["PROPN", "NOUN"]}, "OP": "+"}
]

# Pattern 2 : "abbaye de [Lieu]"
pattern_abbaye = [
    {"LOWER": {"IN": ["abbaye", "monastère", "couvent"]}},
    {"LOWER": {"IN": ["de", "du", "d'"]}},
    {"POS": "PROPN", "OP": "+"}
]

# Pattern 3 : "[Adj] + ien/ais/ois" (gentilés)
# Note : pattern simple, nécessite amélioration
pattern_gentile = [
    {"TEXT": {"REGEX": "^[A-Z][a-z]+iens?$|^[A-Z][a-z]+ais$|^[A-Z][a-z]+ois$"}}
]

matcher.add("RUE", [pattern_rue])
matcher.add("ABBAYE", [pattern_abbaye])
matcher.add("GENTILE", [pattern_gentile])

def extraire_avec_regles(texte):
    """Extraction combinant NER standard et règles personnalisées"""
    doc = nlp_spacy_md(texte)

    # Entités NER standard
    entites_ner = [(ent.text, ent.label_, "NER") for ent in doc.ents
                    if ent.label_ in ["LOC", "GPE"]]

    # Entités via patterns
    matches = matcher(doc)
    entites_rules = []

    for match_id, start, end in matches:
        span = doc[start:end]
        rule_name = nlp_spacy_md.vocab.strings[match_id]
        entites_rules.append((span.text, rule_name, "RULE"))

    return entites_ner, entites_rules

# Test sur exemples problématiques
textes_test = [
    "Il habitait rue de la Paix dans le quartier du Marais.",
    "L'abbaye de Cluny rayonnait sur toute la Bourgogne.",
    "Les Parisiens manifestèrent contre les Versaillais."
]

print("\n🔧 Test avec règles personnalisées :\n")
for texte in textes_test:
    print(f"Texte : {texte}")
    ner, rules = extraire_avec_regles(texte)

    print(f"  NER standard : {[e[0] for e in ner]}")
    print(f"  Règles       : {[(e[0], e[1]) for e in rules]}")
    print()



🔧 Test avec règles personnalisées :

Texte : Il habitait rue de la Paix dans le quartier du Marais.
  NER standard : ['rue de la Paix', 'Marais']
  Règles       : [('Marais', 'GENTILE')]

Texte : L'abbaye de Cluny rayonnait sur toute la Bourgogne.
  NER standard : ['abbaye de Cluny', 'Bourgogne']
  Règles       : [('abbaye de Cluny', 'ABBAYE')]

Texte : Les Parisiens manifestèrent contre les Versaillais.
  NER standard : ['Parisiens', 'Versaillais']
  Règles       : [('Parisiens', 'GENTILE'), ('Versaillais', 'GENTILE')]




## Pipeline complet avec validation


In [8]:
def pipeline_geoparsing_robuste(texte, seuil_confiance=2):
    """
    Pipeline complet de geoparsing avec vote entre outils

    Args:
        texte: texte à analyser
        seuil_confiance: nombre minimum d'outils devant détecter un lieu

    Returns:
        liste de lieux avec score de confiance
    """

    # Étape 1 : Extraction avec tous les outils
    spacy_md = analyser_avec_spacy(texte, nlp_spacy_md)
    stanza = analyser_avec_stanza(texte)

    if nlp_spacy_lg:
        spacy_lg = analyser_avec_spacy(texte, nlp_spacy_lg)
    else:
        spacy_lg = []

    # Étape 2 : Comptage des votes
    compteur_votes = defaultdict(lambda: {'count': 0, 'sources': []})

    for lieu in spacy_md:
        compteur_votes[lieu['texte']]['count'] += 1
        compteur_votes[lieu['texte']]['sources'].append('spaCy-md')

    for lieu in spacy_lg:
        if lieu['texte'] not in [l['texte'] for l in spacy_md]:  # Éviter double comptage
            compteur_votes[lieu['texte']]['count'] += 1
        compteur_votes[lieu['texte']]['sources'].append('spaCy-lg')

    for lieu in stanza:
        compteur_votes[lieu['texte']]['count'] += 1
        compteur_votes[lieu['texte']]['sources'].append('Stanza')

    # Étape 3 : Filtrage par confiance
    lieux_valides = []

    for lieu, info in compteur_votes.items():
        if info['count'] >= seuil_confiance:
            lieux_valides.append({
                'lieu': lieu,
                'confiance': info['count'],
                'sources': info['sources']
            })

    # Tri par confiance décroissante
    lieux_valides.sort(key=lambda x: x['confiance'], reverse=True)

    return lieux_valides

# Test du pipeline
print("="*70)
print("PIPELINE COMPLET AVEC VOTE ENSEMBLISTE")
print("="*70)

texte_complexe = """
En 1804, Napoléon Bonaparte fut sacré empereur à Notre-Dame de Paris.
Ses armées conquirent Vienne, Berlin et Moscou. Les Parisiens célébrèrent
ses victoires, tandis que les cours européennes tremblaient. Le traité de
Tilsit fut signé entre la France et la Russie sur un radeau au milieu du Niémen.
"""

print(f"\nTexte : {texte_complexe}\n")

resultats_pipeline = pipeline_geoparsing_robuste(texte_complexe, seuil_confiance=2)

print("Lieux détectés avec haute confiance :\n")
for res in resultats_pipeline:
    etoiles = "⭐" * res['confiance']
    print(f"{etoiles} {res['lieu']}")
    print(f"   Confiance : {res['confiance']}/3 outils")
    print(f"   Sources : {', '.join(res['sources'])}\n")

PIPELINE COMPLET AVEC VOTE ENSEMBLISTE

Texte : 
En 1804, Napoléon Bonaparte fut sacré empereur à Notre-Dame de Paris.
Ses armées conquirent Vienne, Berlin et Moscou. Les Parisiens célébrèrent
ses victoires, tandis que les cours européennes tremblaient. Le traité de
Tilsit fut signé entre la France et la Russie sur un radeau au milieu du Niémen.


Lieux détectés avec haute confiance :

⭐⭐ Notre-Dame de Paris
   Confiance : 2/3 outils
   Sources : spaCy-md, spaCy-lg, Stanza

⭐⭐ Vienne
   Confiance : 2/3 outils
   Sources : spaCy-md, spaCy-lg, Stanza

⭐⭐ Berlin
   Confiance : 2/3 outils
   Sources : spaCy-md, spaCy-lg, Stanza

⭐⭐ Moscou
   Confiance : 2/3 outils
   Sources : spaCy-md, spaCy-lg, Stanza

⭐⭐ France
   Confiance : 2/3 outils
   Sources : spaCy-md, Stanza

⭐⭐ Russie
   Confiance : 2/3 outils
   Sources : spaCy-md, spaCy-lg, Stanza

⭐⭐ Niémen
   Confiance : 2/3 outils
   Sources : spaCy-md, Stanza



# Usages du NER pour le projet ANR TopUrbi

Werner Stangl, Carmen Brando. TopUrbi and Alcedo's Dictionary: what methods and impasses in the textual and cartographic work of interpreting a historical source?. Colloque Représentations et réalités de l’Amérique après 300 ans de colonisation : le dictionnaire des Indes d’Alcedo, Framespa (UMR 5136); EHESS-CNRS CRH (UMR 8558), Nov 2025, Toulouse, France. ⟨hal-05367622⟩

# Exemple : ANALYSE D'UN CORPUS HISTORIQUE


In [9]:
# Corpus d'exemple : extraits de textes historiques français
corpus_historique = [
    """Le 14 juillet 1789, la Bastille fut prise par le peuple de Paris.
    Les insurgés marchèrent depuis le faubourg Saint-Antoine jusqu'à la forteresse.""",

    """Napoléon quitta l'île d'Elbe en février 1815 et débarqua à Golfe-Juan.
    Il remonta vers Paris en passant par Grenoble et Lyon.""",

    """Jeanne d'Arc naquit à Domrémy en Lorraine. Elle libéra Orléans en 1429
    avant de faire couronner Charles VII à Reims.""",

    """Le traité de Verdun en 843 divisa l'empire carolingien entre les trois
    petits-fils de Charlemagne. La Francie occidentale s'étendait de la Bretagne
    jusqu'à la Bourgogne.""",

    """Les cathédrales gothiques fleurirent en Île-de-France : Notre-Dame de Paris,
    Notre-Dame de Chartres, la cathédrale de Reims et celle d'Amiens."""
]

In [10]:
def analyser_corpus_exercice1(corpus):
    """
    Fonction de départ pour analyser un corpus avec les deux outils
    """

    resultats_analyse = {
        'textes': [],
        'comparaisons': []
    }

    print("\n🔍 Analyse du corpus...\n")

    for i, texte in enumerate(corpus, 1):
        print(f"--- Texte {i} ---")
        print(f"{texte[:80]}...")

        # Extraction avec spaCy
        lieux_spacy = analyser_avec_spacy(texte, nlp_spacy_md)
        lieux_spacy_set = {lieu['texte'] for lieu in lieux_spacy}

        # Extraction avec Stanza
        lieux_stanza = analyser_avec_stanza(texte)
        lieux_stanza_set = {lieu['texte'] for lieu in lieux_stanza}

        # Affichage des résultats
        print(f"  spaCy  : {lieux_spacy_set if lieux_spacy_set else '∅'}")
        print(f"  Stanza : {lieux_stanza_set if lieux_stanza_set else '∅'}")

        # Calcul des métriques de base
        accord = lieux_spacy_set & lieux_stanza_set
        desaccord_spacy = lieux_spacy_set - lieux_stanza_set
        desaccord_stanza = lieux_stanza_set - lieux_spacy_set

        if accord:
            print(f"  ✓ Accord : {accord}")
        if desaccord_spacy:
            print(f"  ⚠ Seulement spaCy : {desaccord_spacy}")
        if desaccord_stanza:
            print(f"  ⚠ Seulement Stanza : {desaccord_stanza}")

        print()

        # Stockage pour analyse ultérieure
        resultats_analyse['textes'].append({
            'texte': texte,
            'spacy': lieux_spacy_set,
            'stanza': lieux_stanza_set,
            'accord': accord,
            'total_lieux': len(lieux_spacy_set | lieux_stanza_set)
        })

    return resultats_analyse

# Lancement de l'analyse
resultats_ex1 = analyser_corpus_exercice1(corpus_historique)


🔍 Analyse du corpus...

--- Texte 1 ---
Le 14 juillet 1789, la Bastille fut prise par le peuple de Paris.
    Les insurg...
  spaCy  : {'Paris', 'la Bastille', 'Saint-Antoine'}
  Stanza : {'faubourg Saint-Antoine', 'Paris', 'Bastille'}
  ✓ Accord : {'Paris'}
  ⚠ Seulement spaCy : {'la Bastille', 'Saint-Antoine'}
  ⚠ Seulement Stanza : {'faubourg Saint-Antoine', 'Bastille'}

--- Texte 2 ---
Napoléon quitta l'île d'Elbe en février 1815 et débarqua à Golfe-Juan.
    Il re...
  spaCy  : {'Golfe-Juan', 'Grenoble', "île d'Elbe", 'Paris', 'Lyon'}
  Stanza : {'Golfe-Juan', 'Grenoble', "île d'Elbe", 'Paris', 'Lyon'}
  ✓ Accord : {'Golfe-Juan', 'Grenoble', "île d'Elbe", 'Paris', 'Lyon'}

--- Texte 3 ---
Jeanne d'Arc naquit à Domrémy en Lorraine. Elle libéra Orléans en 1429
    avant...
  spaCy  : {'Reims', 'Lorraine', 'Orléans'}
  Stanza : {'Domrémy', 'Reims', 'Lorraine', 'Orléans'}
  ✓ Accord : {'Orléans', 'Lorraine', 'Reims'}
  ⚠ Seulement Stanza : {'Domrémy'}

--- Texte 4 ---
Le traité de Ve

## STATISTIQUES GLOBALES

In [11]:
print("\n" + "="*80)
print("📊 STATISTIQUES GLOBALES")
print("="*80)

# Calculer le nombre total de lieux détectés par chaque outil
total_spacy = sum(len(r['spacy']) for r in resultats_ex1['textes'])
total_stanza = sum(len(r['stanza']) for r in resultats_ex1['textes'])

print(f"\nTotal lieux détectés :")
print(f"  spaCy  : {total_spacy}")
print(f"  Stanza : {total_stanza}")

# Calculer le taux d'accord entre les outils
total_accord = sum(len(r['accord']) for r in resultats_ex1['textes'])
total_unique = sum(r['total_lieux'] for r in resultats_ex1['textes'])

if total_unique > 0:
    taux_accord = (total_accord / total_unique) * 100
    print(f"\nTaux d'accord : {taux_accord:.1f}%")
    print(f"  Lieux en accord : {total_accord}/{total_unique}")


📊 STATISTIQUES GLOBALES

Total lieux détectés :
  spaCy  : 17
  Stanza : 20

Taux d'accord : 60.9%
  Lieux en accord : 14/23


# Identifier les patterns d'erreurs / QUESTIONS À EXPLORER :

1. Quels types de lieux sont systématiquement manqués par les deux outils ?
   → Indices : regardez les micro-toponymes, les noms composés...

2. Y a-t-il des faux positifs récurrents ?
   → Indices : noms communs, expressions figées...

3. Quel outil semble plus précis ? Plus exhaustif ?
   → Précision = peu de faux positifs
   → Rappel = peu de faux négatifs

4. Comment améliorer la détection ?
   → Pistes : règles personnalisées, dictionnaires, post-traitement...


## Créer une vérité terrain (gold standard) - A COMPLETER

In [12]:
# Annoter manuellement les vrais lieux
# Exemple pour le premier texte :

verite_terrain_exemple = { # A completer
    0: {'Paris', 'Bastille', 'faubourg Saint-Antoine'},  # Texte 1
    1: {'Elbe', 'Golfe-Juan', 'Paris', 'Grenoble', 'Lyon'},  # Texte 2
    2: {'Domrémy', 'Lorraine'},  # Texte 3
    3: {'Verdun', 'Bretagne', 'Bourgogne'},  # Texte 4
    4: {},  # Texte 5 TODO : à completer
}

def calculer_metriques(predictions, verite_terrain):
    """
    Calcule précision, rappel et F1-score

    Args:
        predictions: set de lieux prédits
        verite_terrain: set de vrais lieux

    Returns:
        dict avec precision, rappel, f1
    """
    if not predictions and not verite_terrain:
        return {'precision': 1.0, 'rappel': 1.0, 'f1': 1.0,
                'vrais_positifs': 0, 'faux_positifs': 0, 'faux_negatifs': 0}

    if not predictions:
        return {'precision': 0.0, 'rappel': 0.0, 'f1': 0.0,
                'vrais_positifs': 0, 'faux_positifs': 0, 'faux_negatifs': len(verite_terrain)}

    if not verite_terrain:
        return {'precision': 0.0, 'rappel': 1.0, 'f1': 0.0,
                'vrais_positifs': 0, 'faux_positifs': len(predictions), 'faux_negatifs': 0}

    vrais_positifs = len(predictions & verite_terrain)
    faux_positifs = len(predictions - verite_terrain)
    faux_negatifs = len(verite_terrain - predictions)

    precision = vrais_positifs / (vrais_positifs + faux_positifs) if (vrais_positifs + faux_positifs) > 0 else 0
    rappel = vrais_positifs / (vrais_positifs + faux_negatifs) if (vrais_positifs + faux_negatifs) > 0 else 0
    f1 = 2 * (precision * rappel) / (precision + rappel) if (precision + rappel) > 0 else 0

    return {
        'precision': precision,
        'rappel': rappel,
        'f1': f1,
        'vrais_positifs': vrais_positifs,
        'faux_positifs': faux_positifs,
        'faux_negatifs': faux_negatifs
    }

# Évaluation quantitative avec la vérité terrain
print("\n" + "="*80)
print("📈 ÉVALUATION QUANTITATIVE")
print("="*80)

# Initialisation des totaux
total_metriques_spacy = {'precision': [], 'rappel': [], 'f1': []}
total_metriques_stanza = {'precision': [], 'rappel': [], 'f1': []}

for i, resultat in enumerate(resultats_ex1['textes']):
    if i in verite_terrain_exemple:
        vt = verite_terrain_exemple[i]

        metriques_spacy = calculer_metriques(resultat['spacy'], vt)
        metriques_stanza = calculer_metriques(resultat['stanza'], vt)

        # Stockage pour moyenne globale
        total_metriques_spacy['precision'].append(metriques_spacy['precision'])
        total_metriques_spacy['rappel'].append(metriques_spacy['rappel'])
        total_metriques_spacy['f1'].append(metriques_spacy['f1'])

        total_metriques_stanza['precision'].append(metriques_stanza['precision'])
        total_metriques_stanza['rappel'].append(metriques_stanza['rappel'])
        total_metriques_stanza['f1'].append(metriques_stanza['f1'])

        print(f"\n📄 Texte {i+1}:")
        print(f"  Vérité terrain : {vt}")
        print(f"\n  spaCy:")
        print(f"    Prédictions : {resultat['spacy']}")
        print(f"    Précision : {metriques_spacy['precision']:.2f}")
        print(f"    Rappel    : {metriques_spacy['rappel']:.2f}")
        print(f"    F1-score  : {metriques_spacy['f1']:.2f}")
        print(f"    VP={metriques_spacy['vrais_positifs']}, FP={metriques_spacy['faux_positifs']}, FN={metriques_spacy['faux_negatifs']}")

        print(f"\n  Stanza:")
        print(f"    Prédictions : {resultat['stanza']}")
        print(f"    Précision : {metriques_stanza['precision']:.2f}")
        print(f"    Rappel    : {metriques_stanza['rappel']:.2f}")
        print(f"    F1-score  : {metriques_stanza['f1']:.2f}")
        print(f"    VP={metriques_stanza['vrais_positifs']}, FP={metriques_stanza['faux_positifs']}, FN={metriques_stanza['faux_negatifs']}")

# Moyennes globales
print("\n" + "="*80)
print("📊 MOYENNES GLOBALES")
print("="*80)

if total_metriques_spacy['precision']:
    print("\nspaCy:")
    print(f"  Précision moyenne : {sum(total_metriques_spacy['precision'])/len(total_metriques_spacy['precision']):.2f}")
    print(f"  Rappel moyen      : {sum(total_metriques_spacy['rappel'])/len(total_metriques_spacy['rappel']):.2f}")
    print(f"  F1-score moyen    : {sum(total_metriques_spacy['f1'])/len(total_metriques_spacy['f1']):.2f}")

    print("\nStanza:")
    print(f"  Précision moyenne : {sum(total_metriques_stanza['precision'])/len(total_metriques_stanza['precision']):.2f}")
    print(f"  Rappel moyen      : {sum(total_metriques_stanza['rappel'])/len(total_metriques_stanza['rappel']):.2f}")
    print(f"  F1-score moyen    : {sum(total_metriques_stanza['f1'])/len(total_metriques_stanza['f1']):.2f}")


📈 ÉVALUATION QUANTITATIVE

📄 Texte 1:
  Vérité terrain : {'faubourg Saint-Antoine', 'Paris', 'Bastille'}

  spaCy:
    Prédictions : {'Paris', 'la Bastille', 'Saint-Antoine'}
    Précision : 0.33
    Rappel    : 0.33
    F1-score  : 0.33
    VP=1, FP=2, FN=2

  Stanza:
    Prédictions : {'faubourg Saint-Antoine', 'Paris', 'Bastille'}
    Précision : 1.00
    Rappel    : 1.00
    F1-score  : 1.00
    VP=3, FP=0, FN=0

📄 Texte 2:
  Vérité terrain : {'Elbe', 'Golfe-Juan', 'Grenoble', 'Paris', 'Lyon'}

  spaCy:
    Prédictions : {'Golfe-Juan', 'Grenoble', "île d'Elbe", 'Paris', 'Lyon'}
    Précision : 0.80
    Rappel    : 0.80
    F1-score  : 0.80
    VP=4, FP=1, FN=1

  Stanza:
    Prédictions : {'Golfe-Juan', 'Grenoble', "île d'Elbe", 'Paris', 'Lyon'}
    Précision : 0.80
    Rappel    : 0.80
    F1-score  : 0.80
    VP=4, FP=1, FN=1

📄 Texte 3:
  Vérité terrain : {'Domrémy', 'Lorraine'}

  spaCy:
    Prédictions : {'Reims', 'Lorraine', 'Orléans'}
    Précision : 0.33
    Rappel    : 0.

# Liens utiles

📚 Documentation :
   - spaCy : https://spacy.io/usage/linguistic-features#named-entities
   - Stanza : https://stanfordnlp.github.io/stanza/
   - Prodigy : outil d'annotation (payant mais efficace)
   - PERDIDO : Geo NER, https://github.com/ludovicmoncla/perdido

🗺️ Bases de données géographiques :
   - GeoNames : http://www.geonames.org/
   - Pleiades : https://pleiades.stoa.org/ (Antiquité)
   - Getty TGN : http://www.getty.edu/research/tools/vocabularies/tgn/

📊 Datasets pour l'évaluation :
   - CoNLL-2003 (anglais)
   - WikiNER (multilingue)
   - Quaero (français, incluant lieux)
